In [ ]:
# N Review Crwaler, 구조는 최대한 간결하게 id[] , document[text], label[부정:0 , 긍정:1] 로 한다.
# url sample
#https://pcmap.place.naver.com/restaurant/20709913/review/visitor?entry=pll&from=map&fromPanelNum=2&ts=1623297464681

